In [ ]:
import xml.etree.ElementTree as ET
import re

# Define the input file path
input_file = "/Users/promachowdhury/Downloads/code-metrics (6)/lizard_report.xml"

# Read and clean the XML file
with open(input_file, "r", encoding="utf-8") as f:
    content = f.read()

# Find the starting position of the XML declaration
xml_start = content.find("<?xml version")
if xml_start != -1:
    content = content[xml_start:]

# Parse the XML content
root = ET.fromstring(content)

# Separate test and non-test files
test_metrics = {"Nr": [], "NCSS": [], "CCN": []}
non_test_metrics = {"Nr": [], "NCSS": [], "CCN": []}

# Process each <item> in the XML
for item in root.findall(".//item"):
    name = item.get("name")
    values = [int(val.text) for val in item.findall("value")[:3]] 

    if name and values:
        if re.search(r"\btests?\b", name): 
            test_metrics["Nr"].append(values[0])
            test_metrics["NCSS"].append(values[1])
            test_metrics["CCN"].append(values[2])
        else:  
            non_test_metrics["Nr"].append(values[0])
            non_test_metrics["NCSS"].append(values[1])
            non_test_metrics["CCN"].append(values[2])

def compute_average(metrics):
    return {key: sum(vals) / len(vals) if vals else 0 for key, vals in metrics.items()}


test_averages = compute_average(test_metrics)
non_test_averages = compute_average(non_test_metrics)


print("Test Files Averages:", test_averages)
print("Non-Test Files Averages:", non_test_averages)


Test Files Averages: {'Nr': 264.61538461538464, 'NCSS': 21.573870573870575, 'CCN': 2.6788766788766787}
Non-Test Files Averages: {'Nr': 1063.180339985218, 'NCSS': 64.58610495195862, 'CCN': 7.144124168514413}


In [ ]:
import json

# File path
file_path = "/Users/promachowdhury/Downloads/code-metrics (6)/halstead.json"

# Load the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)


test_values = {}
non_test_values = {}


for file_path, metrics in data.items():
    is_test = "/tests/" in file_path 
    category = test_values if is_test else non_test_values

    for metric, value in metrics["total"].items():
        if metric not in category:
            category[metric] = []
        category[metric].append(value)


def compute_avg(values_dict):
    return {metric: sum(values) / len(values) if values else 0 for metric, values in values_dict.items()}


test_averages = compute_avg(test_values)
non_test_averages = compute_avg(non_test_values)


print("Test Files Averages:", test_averages)
print("Non-Test Files Averages:", non_test_averages)


Test Files Averages: {'h1': 2.36318407960199, 'h2': 20.970149253731343, 'N1': 17.442786069651742, 'N2': 28.577114427860696, 'vocabulary': 23.333333333333332, 'length': 46.01990049751244, 'calculated_length': 140.25828960295004, 'volume': 289.19051389257027, 'difficulty': 1.6826429523514541, 'effort': 1173.0530507260382, 'time': 65.16961392922434, 'bugs': 0.09639683796419021}
Non-Test Files Averages: {'h1': 2.9452054794520546, 'h2': 23.246575342465754, 'N1': 28.1324200913242, 'N2': 42.74429223744292, 'vocabulary': 26.19178082191781, 'length': 70.87671232876713, 'calculated_length': 167.73122775125947, 'volume': 395.22573348395633, 'difficulty': 3.7062815018465995, 'effort': 3649.069801036498, 'time': 202.7261000575831, 'bugs': 0.13174191116131886}


In [3]:
import json

# File path
file_path = "/Users/promachowdhury/Downloads/code-metrics (6)/raw_metrics.json"

# Load the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Initialize dictionaries to store metric values separately for test and non-test files
test_values = {}
non_test_values = {}

# Process each file in the JSON
for file_path, metrics in data.items():
    is_test = "/tests/" in file_path  # Identify test files
    category = test_values if is_test else non_test_values

    for metric, value in metrics.items():
        if metric not in category:
            category[metric] = []
        category[metric].append(value)

# Function to compute averages
def compute_avg(values_dict):
    return {metric: sum(values) / len(values) if values else 0 for metric, values in values_dict.items()}

# Compute separate averages
test_averages = compute_avg(test_values)
non_test_averages = compute_avg(non_test_values)

# Print results
print("Test Files Averages:", test_averages)
print("Non-Test Files Averages:", non_test_averages)


Test Files Averages: {'loc': 77.30845771144278, 'lloc': 34.35820895522388, 'sloc': 48.35820895522388, 'comments': 17.62686567164179, 'multi': 0.0, 'blank': 11.472636815920398, 'single_comments': 17.47761194029851}
Non-Test Files Averages: {'loc': 200.324200913242, 'lloc': 86.70319634703196, 'sloc': 147.6027397260274, 'comments': 17.904109589041095, 'multi': 0.0, 'blank': 35.67579908675799, 'single_comments': 17.04566210045662}


In [5]:
import json
from typing import Dict, List, Union

def calculate_complexity_metrics(data: Dict[str, List[Dict]]) -> Dict[str, float]:
    """
    Calculate average complexity metrics separately for test and non-test files.

    Args:
        data: Dictionary with filenames as keys and lists of code elements as values.

    Returns:
        Dictionary containing separate complexity metrics for test and non-test files.
    """
    test_methods, test_classes = [], []
    non_test_methods, non_test_classes = [], []

    # Collect complexity values separately for test and non-test files
    for filename, elements in data.items():
        is_test_file = "/tests/" in filename  # Identify test files
        
        for element in elements:
            if element["type"] in ["function", "method"]:
                if is_test_file:
                    test_methods.append(element["complexity"])
                else:
                    non_test_methods.append(element["complexity"])
            elif element["type"] == "class":
                if is_test_file:
                    test_classes.append(element["complexity"])
                else:
                    non_test_classes.append(element["complexity"])

    # Function to compute complexity stats
    def compute_metrics(methods: List[int], classes: List[int]) -> Dict[str, Union[float, int]]:
        return {
            "average_method_complexity": sum(methods) / len(methods) if methods else 0,
            "average_class_complexity": sum(classes) / len(classes) if classes else 0,
            "total_methods": len(methods),
            "total_classes": len(classes),
            "max_method_complexity": max(methods) if methods else 0,
            "min_method_complexity": min(methods) if methods else 0,
        }

    return {
        "test_files": compute_metrics(test_methods, test_classes),
        "non_test_files": compute_metrics(non_test_methods, non_test_classes),
    }

def main():
    # Load the JSON file
    with open('/Users/promachowdhury/Downloads/code-metrics (6)/complexity.json', 'r') as f:
        data = json.load(f)

    try:
        # Calculate metrics
        metrics = calculate_complexity_metrics(data)

        # Print results
        print(f"\nCode Complexity Analysis Results (Test Files):")
        print(f"----------------------------------------------")
        print(f"Average Method Complexity: {metrics['test_files']['average_method_complexity']:.2f}")
        print(f"Average Class Complexity: {metrics['test_files']['average_class_complexity']:.2f}")
        print(f"Total Methods: {metrics['test_files']['total_methods']}")
        print(f"Total Classes: {metrics['test_files']['total_classes']}")
        print(f"Highest Method Complexity: {metrics['test_files']['max_method_complexity']}")
        print(f"Lowest Method Complexity: {metrics['test_files']['min_method_complexity']}")

        print(f"\nCode Complexity Analysis Results (Non-Test Files):")
        print(f"--------------------------------------------------")
        print(f"Average Method Complexity: {metrics['non_test_files']['average_method_complexity']:.2f}")
        print(f"Average Class Complexity: {metrics['non_test_files']['average_class_complexity']:.2f}")
        print(f"Total Methods: {metrics['non_test_files']['total_methods']}")
        print(f"Total Classes: {metrics['non_test_files']['total_classes']}")
        print(f"Highest Method Complexity: {metrics['non_test_files']['max_method_complexity']}")
        print(f"Lowest Method Complexity: {metrics['non_test_files']['min_method_complexity']}")

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()



Code Complexity Analysis Results (Test Files):
----------------------------------------------
Average Method Complexity: 4.23
Average Class Complexity: 6.67
Total Methods: 624
Total Classes: 6
Highest Method Complexity: 221
Lowest Method Complexity: 1

Code Complexity Analysis Results (Non-Test Files):
--------------------------------------------------
Average Method Complexity: 4.34
Average Class Complexity: 5.20
Total Methods: 1098
Total Classes: 203
Highest Method Complexity: 78
Lowest Method Complexity: 1
